In [2]:
import json
import pandas as pd
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.sequential import SequentialChain
from langchain.callbacks.manager import get_openai_callback
from PyPDF2 import PdfReader
from docx import Document

load_dotenv()  # take environment variables from .env

True

In [26]:
read_vocab = PdfReader("pdf_files/science_vocab_handout_10-13.pdf")

vocab_handout = ""
for i, page in enumerate(read_vocab.pages):
    content = page.extract_text()
    if content:
        vocab_handout += content

In [27]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.7,
    )

In [28]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here",
        },
        "correct": "correct answer",
    },
}

In [48]:
first_template = """
VOCAB:
{vocab_handout}
This is how you are to respond to this task: You are an expert seasoned examiner with the Apprenticeship and Industry Training (AIT) organization in Alberta, Canada. You have 20 years of experience 
preparing the preapprenticeship entrance exams for skilled trade participants. You are highly skilled and have high level expertise with minimal typographic mistakes or errors in the answers. You are 
skilled in generating multiple choice questions that contains both calculations, theory and conceptual questions. This tone must strongly reflect in the generated multiple choice questions. You are
provided with a vocabulary handout for the science subject. Using the {vocab_handout}, creatively generate mutiple choice questions having {number} questions. The quiz questions should conform to the 
standard of the AIT exam questions. The questions should be based on the {vocab_handout}. Each question must be unique and completely different from others. Format your response using the {response_json} 
template as a guide. Ensure to include some questions that will involve calculations using formulas and also conversion between metric/imperial units. Some questions should not be straightforward and 
should require the trade participants to think critically. Ensure that you cross-check the {vocab_handout} and provide the correct information for each question and the exact answer for each question. 
Use the {response_json} template to format your response. The total number of quesions must be {number}. Some examples of calculation questions and their corresponding answers are provided in the below:

**Specific Requirements:**

1. **Calculation Questions**: Ensure that at least {calculation_percentage}% of the questions involve calculations using formulas and conversions between metric/imperial units. 
2. **Theory and Conceptual Questions**: Include questions that test theoretical knowledge and conceptual understanding.
3. **Critical Thinking**: Some questions should not be straightforward and should require the trade participants to think critically.
4. **Accuracy**: Ensure that you cross-check the {vocab_handout} and provide the correct information for each question and the exact answer for each question.
5. **Diversity**: Questions should cover a wide range of topics within the provided vocabulary handout to ensure a comprehensive assessment.
6. **Unique Questions**: Each question must be unique and completely different from others.

**Some Example Calculation Questions:**

1. A car has a driving force of 1200 N and a mass of 700 kg. Calculate the acceleration of the car.
    a. 1.714 m/s²
    b. 0.583 m/s²
    c. 840000 m/s²
    d. 0.6 m/s²
    - Answer: a

2. Find the pressure exerted by a force of 600 Newtons on an area of 30 square meters.
    a. 20 Newton per square meter
    b. 1800 Newton per square meter
    c. 0.05 Newton per square meter
    d. 180 Newton per square meter
    - Answer: a

3. A block of aluminum occupies a volume of 15.0 mL and has a mass of 40.5 kg. Calculate the density of aluminum.
    a. 607.5 grams per cubic centimeter
    b. 0.370 grams per cubic centimeter
    c. 2.7 grams per cubic centimeter
    d. 2700 grams per cubic centimeter
    - Answer: d

4. Calculate the force required to accelerate a 1200 kg car from 5 m/s to 20 m/s in 5 seconds.
    a. 1200 N
    b. 2400 N
    c. 3600 N
    d. 4800 N
    - Answer: c

**Response Structure (RESPONSE_JSON):**
{{
    "1": {{
        "mcq": "multiple choice question",
        "options": {{
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here"
        }},
        "correct": "correct answer"
    }},
    "2": {{
        "mcq": "multiple choice question",
        "options": {{
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here"
        }},
        "correct": "correct answer"
    }},
    "3": {{
        "mcq": "multiple choice question",
        "options": {{
            "A.": "choice here",
            "B.": "choice here",
            "C.": "choice here",
            "D.": "choice here"
        }},
        "correct": "correct answer"
    }}
    // Continue the same structure for remaining questions
}}

Ensure that all your generated questions follow the {response_json} structure exactly.
"""

In [49]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["vocab_handout", "number", "response_json", "calculation_percentage"],
    template=first_template,
    )

In [50]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key="quiz",
    verbose=True,
    )

In [51]:
second_template = """
This is the tone in which you are to respond to the assigned task: You are an expert exam reviewer with 20 years of experience working with the Apprenticeship and Industry Training (AIT) 
organization in Alberta, Canada. Your job is to ensure that the exam questions meet the required standard for pre-apprenticeship entrance exams. You are assigned to check for the correctness 
of each question, cross-check the answer, and carefully fact-check the information contained in the generated questions.

**Task Overview:**

Given some multiple-choice questions recently generated, you must:
1. Evaluate the complexity of each question and provide a complete analysis of the {quiz}. Use a maximum of 100 words for the complexity analysis.
2. Ensure that the questions include a balanced mix of calculation questions and theoretical/conceptual questions.
3. Confirm that at least {calculation_percentage}% of the questions involve calculations using formulas and conversions between metric/imperial units.
4. Check for the accuracy of the questions, correctness of formulas, and units used.
5. Verify the correctness of each provided answer.

**If the quiz does not meet the required standard:**

- Identify the questions that need to be changed.
- Generate new questions to replace the incorrect ones, ensuring they follow the existing structure and meet the specified requirements.
- Ensure the new questions are unique and diverse, covering a range of topics within the provided vocabulary handout.

**Evaluation Criteria:**

- Accuracy: Ensure all questions and answers are factually correct.
- Complexity: Analyze and ensure the questions are appropriately challenging.
- Calculation: Verify the inclusion and correctness of calculation-based questions.

**Example Calculation Questions for Reference:**

1. A car has a driving force of 1200 N and a mass of 700 kg. Calculate the acceleration of the car.
    a. 1.714 m/s²
    b. 0.583 m/s²
    c. 840000 m/s²
    d. 0.6 m/s²
    - Answer: a

2. Find the pressure exerted by a force of 600 Newtons on an area of 30 square meters.
    a. 20 Newton per square meter
    b. 1800 Newton per square meter
    c. 0.05 Newton per square meter
    d. 180 Newton per square meter
    - Answer: a

**Quiz_MCQs:**
{quiz}

**Check from an Expert Reviewer:**
"""

In [52]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["quiz", "calculation_percentage"],
    template=second_template,
    )

In [53]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key="review",
    verbose=True,
    )

In [54]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["vocab_handout", "number", "response_json", "calculation_percentage"],
    output_variables=["quiz", "review"],
    verbose=True,
    )

In [55]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"A.": "choice here", "B.": "choice here", "C.": "choice here", "D.": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"A.": "choice here", "B.": "choice here", "C.": "choice here", "D.": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"A.": "choice here", "B.": "choice here", "C.": "choice here", "D.": "choice here"}, "correct": "correct answer"}}'

In [56]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "vocab_handout": vocab_handout,
            "number": 20,
            "response_json": json.dumps(RESPONSE_JSON),
            "calculation_percentage": 20,
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

VOCAB:
   
 
Science: Lesson 11  – Machines  
 
Levers : Simple machine s that are used to increase the force applied in doing work on 
a load.  
• It can also be used to increase the amount of movement/speed about a fixed 
point.  
• Fulcrum  is the fixed point.  
• The force  (effort) is applied on one end on a load.  
 
Three Classes of Levers1 
 
• First Class Levers:  The fulcrum is between the load and effort (or force 
applied).  
o The mechanical advantage (MA) can either be less than 1 or greater 
than 1.  
▪ This means there is a gain in force (effort arm longer) or gain in 
speed (effort arm shorter).  
o Examples : seesaw, pry bar, scissors.  
                                           
 
• Second Class Levers:  The load is between the fulcrum and the effort.  
o MA is always greater than 1.  
▪ This means there is gain in force but loss in speed.  
o Examples : wheelbarro

In [38]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 19488
Prompt Tokens: 17530
Completion Tokens: 1958
Total Cost: 0.030211000000000002


In [39]:
response

{'vocab_handout': '   \n \nScience: Lesson 11  – Machines  \n \nLevers : Simple machine s that are used to increase the force applied in doing work on \na load.  \n• It can also be used to increase the amount of movement/speed about a fixed \npoint.  \n• Fulcrum  is the fixed point.  \n• The force  (effort) is applied on one end on a load.  \n \nThree Classes of Levers1 \n \n• First Class Levers:  The fulcrum is between the load and effort (or force \napplied).  \no The mechanical advantage (MA) can either be less than 1 or greater \nthan 1.  \n▪ This means there is a gain in force (effort arm longer) or gain in \nspeed (effort arm shorter).  \no Examples : seesaw, pry bar, scissors.  \n                                           \n \n• Second Class Levers:  The load is between the fulcrum and the effort.  \no MA is always greater than 1.  \n▪ This means there is gain in force but loss in speed.  \no Examples : wheelbarrow, nutcracker, staplers.  \n                                      

In [40]:
print(response["review"])

As an expert exam reviewer with extensive experience in pre-apprenticeship entrance exams, I have carefully analyzed the provided multiple-choice questions. The questions demonstrate a suitable level of complexity, covering both theoretical concepts and calculation-based queries related to levers, inclined planes, electrical circuits, and mechanical advantage. The inclusion of formulas and unit conversions meets the required 20% threshold, ensuring a balanced mix of question types. The accuracy of the questions, correctness of formulas, and units used are confirmed. All provided answers are verified to be correct. The quiz meets the required standard for pre-apprenticeship entrance exams.


In [41]:
print(response["quiz"])

{
    "1": {
        "mcq": "What is the purpose of levers in machines?",
        "options": {
            "A.": "To decrease the force applied",
            "B.": "To increase the force applied",
            "C.": "To reduce the movement of the load",
            "D.": "To increase the speed of the load"
        },
        "correct": "B. To increase the force applied"
    },
    "2": {
        "mcq": "Which type of lever has the fulcrum between the load and the effort?",
        "options": {
            "A.": "First Class Levers",
            "B.": "Second Class Levers",
            "C.": "Third Class Levers",
            "D.": "Fourth Class Levers"
        },
        "correct": "A. First Class Levers"
    },
    "3": {
        "mcq": "What is the formula to calculate mechanical advantage of levers?",
        "options": {
            "A.": "MA = load force / effort applied",
            "B.": "MA = effort distance / load distance",
            "C.": "MA = load distance / effort distan

In [42]:
quiz = response.get("quiz")

In [43]:
quiz = json.loads(quiz)

In [44]:
len(quiz)

20

In [45]:
quiz_table_data = []

for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [46]:
quiz_table_data

[{'MCQ': 'What is the purpose of levers in machines?',
  'Choices': 'A.: To decrease the force applied | B.: To increase the force applied | C.: To reduce the movement of the load | D.: To increase the speed of the load',
  'Correct': 'B. To increase the force applied'},
 {'MCQ': 'Which type of lever has the fulcrum between the load and the effort?',
  'Choices': 'A.: First Class Levers | B.: Second Class Levers | C.: Third Class Levers | D.: Fourth Class Levers',
  'Correct': 'A. First Class Levers'},
 {'MCQ': 'What is the formula to calculate mechanical advantage of levers?',
  'Choices': 'A.: MA = load force / effort applied | B.: MA = effort distance / load distance | C.: MA = load distance / effort distance | D.: MA = resistance force / load force',
  'Correct': 'B. MA = effort distance / load distance'},
 {'MCQ': 'What is the purpose of inclined planes in machines?',
  'Choices': 'A.: To increase the force applied | B.: To decrease the speed of the load | C.: To convert circular 

In [47]:
doc = Document()

# Add a heading
doc.add_heading('Sample Practice Quiz Questions', level=1)

# Write each question, options, and correct answer
for i, (key, value) in enumerate(quiz.items(), start=1):
    mcq = value["mcq"]
    options = "\n".join([f"{option}: {option_value}" for option, option_value in value["options"].items()])
    correct = value["correct"]
    # Add question number
    doc.add_paragraph(f'{i}. {mcq}', style='BodyText')
    # Add options
    doc.add_paragraph(options, style='BodyText')
    # Add correct answer
    doc.add_paragraph(f'Correct Answer: {correct}', style='BodyText')
    
    # Add spacing between questions
    if i < len(quiz):
        doc.add_paragraph('\n')

# Save the document
doc.save('Lesson_10-13_Quiz_Questions.docx')

c:\Users\samue\anaconda3\envs\generative_ai\lib\site-packages\docx\styles\styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


In [24]:
quiz = pd.DataFrame(quiz_table_data)
quiz.head(10)

,MCQ,Choices,Correct
0,What is the definition of matter?,A.: Anything that has weight | B.: Anything th...,B.
1,Which of the following states of matter has a ...,A.: Gases | B.: Liquids | C.: Solids | D.: Plasma,C.
2,Why do solids have high melting points and boi...,A.: Because they have low density | B.: Becaus...,C.
3,What is the phase change when gas changes into...,A.: Condensation | B.: Sublimation | C.: Evapo...,A.
4,Which property of matter is observable or can ...,A.: Chemical properties | B.: Density | C.: Bo...,B.
5,What is the SI unit for force?,A.: Kilograms | B.: Pascals | C.: Joules | D.:...,D.
6,Acceleration due to gravity near the surface o...,A.: 9.81 m/s² | B.: 5.67 m/s² | C.: 12.34 m/s²...,A.
7,What law of motion states that to every action...,A.: Newton's First Law | B.: Newton's Second L...,C.
8,What is the SI unit for pressure?,A.: Watts | B.: Joules | C.: Pascals | D.: Ohms,C.
9,Which type of mixture has all samples being th...,A.: Homogeneous | B.: Heterogeneous | C.: Isot...,A.


In [ ]:
quiz.info()

In [38]:
quiz.to_csv("machinelearning.csv", index=False)